<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer: Receives input, data that is exposed, correlates to number of features
### Hidden Layer: Anything between input and output, data cannot be seen. Inside network they perform function
### The function is learnt
### Output Layer: Outputs vector values for whatever you need to address, helps with activation value one can ###interpret
### Single output, no activation predicts regression
### Sigmoid function represents the binary classification's probability
### one node for each item one tries to predict - Multi-class classification problems
### multi-target regression- one node for each regression prediction
### Neuron: a variable, something that holds a value
### Weight: parameter
### Activation Function: Node
### Node Map: Nodes are for each item one is trying to predict
### Perceptron:A basic neural network with a input and output cell
### Bias: number to add on before squishification - to make it to be more active or inactive -
### tells you how high the weighted sum needs to be before the neuron gets meaningfully active.
### we add all biases into a vector and add the vector into the previous matrix vector product
### Sigmoid: squishification function, which is applied to each existing component
### Deep learning: more hidden layers

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Your Answer Here

## Write your own perceptron code that can correctly classify a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [ ]:
##### Your Code Here #####

## Implement your own Perceptron Class and use it to classify a binary dataset like: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 
- [Titanic](https://raw.githubusercontent.com/ryanleeallred/datasets/master/titanic.csv)
- [A two-class version of the Iris dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Iris.csv)

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [ ]:
##### Your Code Here #####

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?

In [ ]:
#backpropagation requires a lot of training data - how a single training example should be nudged to have the most rapid decrease to the cost function
#is the algorithmn that adds all the changes in weights and biases over all output neurons 
#in the second to last layer to get less error
#these changes are applied to weights and biases that determine the second to last layer, and the layer before that
#the backpropagation routine is done for every other training example - recording how each would like to change weights and biases
#the changes are the negative gradient of the cost function

#often training data is divided into mini-batches, a mini batch gives an approximation - stochastic gradient descent
